In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("HuggingFaceTB/Countdown-Task-GOLD", "verified_Qwen3-4B-Instruct-2507")

for split, dataset in ds.items():
    dataset.to_json(f"{split}.jsonl", orient="records", lines=True)

Creating json from Arrow format: 100%|██████████| 28/28 [00:00<00:00, 57.92ba/s]


In [5]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("HuggingFaceTB/Countdown-Task-GOLD", "test")

for split, dataset in ds.items():
    dataset.to_json(f"test.jsonl", orient="records", lines=True)

Creating json from Arrow format: 100%|██████████| 10/10 [00:00<00:00, 170.38ba/s]


In [6]:
from datasets import load_dataset

path = "/data/shanghong/oumi/gold/train_verified_Qwen3-4B-Instruct-2507.jsonl"

# Load JSONL
ds = load_dataset("json", data_files=path, split="train")

# 90/10 split
split_ds = ds.train_test_split(test_size=0.1, seed=42)

train_ds = split_ds["train"]
val_ds = split_ds["test"]

print(train_ds, val_ds)

train_ds.to_json(
    "/data/shanghong/oumi/gold/train_90.jsonl",
    orient="records",
    lines=True
)

val_ds.to_json(
    "/data/shanghong/oumi/gold/val_10.jsonl",
    orient="records",
    lines=True
)


Generating train split: 27667 examples [00:00, 235869.07 examples/s]


Dataset({
    features: ['target', 'nums', 'messages'],
    num_rows: 24900
}) Dataset({
    features: ['target', 'nums', 'messages'],
    num_rows: 2767
})


Creating json from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 40.00ba/s]


5620932

In [3]:
from datasets import load_dataset

path = "/data/shanghong/oumi/gold/data/train_90.jsonl"

# Load JSONL
ds = load_dataset("json", data_files=path, split="train")

# Shuffle for randomness
ds = ds.shuffle(seed=42)

# Take a small subset
train_ds = ds.select(range(128))   # change 128 → 256 / 512 as needed

print(train_ds)

# Save
train_ds.to_json(
    "/data/shanghong/oumi/gold/data/train_small_128.jsonl",
    orient="records",
    lines=True
)


Generating train split: 24900 examples [00:00, 223603.30 examples/s]


Dataset({
    features: ['target', 'nums', 'messages'],
    num_rows: 128
})


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 161.22ba/s]


262786

In [2]:
import json

input_path = "/data/shanghong/oumi/gold/output/qwen2.5_1.5b_ckpt500.json"
test_path = "/data/shanghong/oumi/gold/data/test.jsonl"
output_path = "/data/shanghong/oumi/gold/output/qwen2.5_1.5b_ckpt500.jsonl"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_data = []
with open(test_path, "r") as f:
    for line in f:
        test_data.append(json.loads(line))

for item, test_item in zip(data, test_data):
    item["target"] = test_item["target"]
    item["nums"] = test_item["nums"]


with open(output_path, "w", encoding="utf-8") as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
